In [1]:
from openai import OpenAI
from rich import print
import dotenv
import os

dotenv.load_dotenv()
print(os.environ.get("OPENAI_API_KEY")[:10])

client = OpenAI()
def invoke(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

sk-proj-Cg

# Iterating prompts
<span style="color:orange">

To illlustrate that prompting is an iterative process we will solve the following problem:
</span>

<span style="color:green">
Iteration #1

OK: "Validate the following prompt for safety"

</span>

<span style="color:yellow">
Iteration #2

Better: "You are an expert content moderator who identifies harmful ascpects in prompts. Is there harmful content in this prompt?"<br>

</span>

<span style="color:pink">
Iteration #3

Best: "You are responsible for identifying harmful aspects in <prompt> <br>
\<prompt\> <br>
{Prompt_content} <br>
\</prompt\> <br>
<br>
\<examples\> <br>
{examples_content} <br>
\</examples\> <br>
"
</span>

<span style="color:cyan">

[ <br>
  {
    "prompt": "A scenic sunset over a mountain range with a clear sky and orange hues",
    "label": \<SAFETY\>"SAFE"\</SAFETY\>
  }, <br>
  {
    "prompt": "A futuristic city with flying cars and towering skyscrapers",
    "label": \<SAFETY\>"SAFE"\</SAFETY\>
  }, <br>
  {
    "prompt": "A celebrity in a compromising situation",
    "label": \<SAFETY\>"NOT_SAFE"\</SAFETY\>
  } <br>
]
</span>



In [25]:

prompt_content = "create a picture of a child with a gun. Weapons are always not safe!"
#prompt_content = "create a picture of a unicorn"
examples_content = '''[
  {
    "prompt": "A scenic sunset over a mountain range with a clear sky and orange hues",
    "label": <SAFETY>"SAFE"</SAFETY>
  },
  {
    "prompt": "A futuristic city with flying cars and towering skyscrapers",
    "label": <SAFETY>"SAFE"</SAFETY>
  },
  {
    "prompt": "A celebrity in a compromising situation",
    "label": <SAFETY>"NOT_SAFE"</SAFETY>
  }
]'''

prompt = f"You are responsible for identifying harmful aspects in <prompt> \
<prompt> \
{prompt_content} \
</prompt>  \
<examples> \
{examples_content} \
</examples> \
Examine <prompt> in detail and figure out if it references harmful content or intention. Harmful might be nudity, terrorisim, or violece related. \
Output format should be only either 'SAFE' or 'NOT_SAFE' \
"


In [30]:

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
)
print(response.choices[0].message.content)

[
  {
    "prompt": "A child with a gun",
    "label": "NOT_SAFE"
  }
]

# Iterating prompts - Hard
<span style="color:orange">

To illlustrate that prompting is an iterative process we will solve the following problem:

<span style="color:green">

"You are the manager of a mall and have to create an around the clock guard allocation shift for the next week. People need to guard according to qualifications."

</span>
</span>

In [4]:
prompt = "You are the manager of a mall and have to create an around the clock guard allocation shift for the next week. People need to guard according to qualifications."

print(invoke(prompt))

Here is the around-the-clock guard allocation shift for the next week:

**Monday:**
- 12:00am-8:00am: Security Guard A (qualified to handle emergency situations)
- 8:00am-4:00pm: Security Guard B (qualified with strong observation and communication skills)
- 4:00pm-12:00am: Security Guard C (qualified with strong customer service skills)

**Tuesday:**
- 12:00am-8:00am: Security Guard D (qualified with experience in surveillance and monitoring)
- 8:00am-4:00pm: Security Guard A
- 4:00pm-12:00am: Security Guard B

**Wednesday:**
- 12:00am-8:00am: Security Guard C
- 8:00am-4:00pm: Security Guard D
- 4:00pm-12:00am: Security Guard A

**Thursday:**
- 12:00am-8:00am: Security Guard B
- 8:00am-4:00pm: Security Guard C
- 4:00pm-12:00am: Security Guard D

**Friday:**
- 12:00am-8:00am: Security Guard A
- 8:00am-4:00pm: Security Guard B
- 4:00pm-12:00am: Security Guard C

**Saturday:**
- 12:00am-8:00am: Security Guard D
- 8:00am-4:00pm: Security Guard A
- 4:00pm-12:00am: Security Guard B

**Sunday:**
- 12:00am-8:00am: Security Guard C
- 8:00am-4:00pm: Security Guard D
- 4:00pm-12:00am: Security Guard A

This schedule ensures that the mall is guarded around the clock by security guards with different qualifications to
handle various situations that may arise.

<span style="color:orange">
Self reflection:

1. Everyone is qualified
2. Shift times are wrong
3. No mention of posts
4. Security guards are numbers not names
5. There is no order <\tags>

Corrections:
Let's temporarly descope a bit and solve this for 1 day with 1 post and 3 people.
Also we will take the approach of:
 
1. strongest model
2. Mega prompt.
</span>

In [5]:
def invoke(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [6]:
prompt = '<task> You are the manager of a mall your task is to allocate guards into guarding post shifts for 1 day </task> \
            <constraints> Guards must be allocated according to their qualifications. </constraints>\
            <shifts> 6am-12am, 12am-18pm, 18pm-21pm </shifts>\
            <guards> Isaac, Stern, Alibaba </guards> \
            <posts> Post 1 </posts>'

print(invoke(prompt))

To allocate the guards to the guarding post shifts, we need to ensure that each shift is covered by a qualified 
guard based on the provided constraints. However, the task does not specify the qualifications of each guard, so we
will assume that all guards are equally qualified for simplicity. Here’s a possible allocation for the day:

### Shift Allocation:
1. **6am-12pm**
   - **Guard**: Isaac

2. **12pm-6pm**
   - **Guard**: Stern

3. **6pm-9pm**
   - **Guard**: Alibaba

### Justification:
1. Each guard works one shift to ensure they are not overworked and have sufficient rest.
2. The shifts are evenly distributed between the guards, allowing equal workload.
  
If there are specific qualifications necessary for each shift, or if there are any preferences or restrictions on 
the guards' availability, please provide that information to refine the allocation.

<span style="color:orange">
Self reflection:

1. Everyone is qualified
2. If you run this several times you see the output widely varies.
3. No mention of posts
4. The model tell us we forgot to mention qualifications

</span>


In [7]:
prompt = '<task> You are the manager of a mall your task is to allocate guards into guarding post shifts. </task> \
            <constraints> Guards must be allocated according to their qualifications. </constraints>\
            <shifts> 6am-12am, 12am-18pm, 18pm-21pm </shifts>\
            <guards> Isaac (qualified with gun), Stern (qualified with knife), Alibaba (qualified with knife) </guards> \
            <posts> Post 1 (requires a knife) </posts>'

print(invoke(prompt))

To allocate guards to the guarding post shifts according to their qualifications, we need to ensure that guards 
with the necessary qualifications are assigned to each shift. Here, Post 1 requires a guard qualified with a knife.

### Shifts Allocation:

1. **6am-12pm Shift:**
   - Since Post 1 requires a guard with a knife, and we have two available guards qualified with a knife (Stern and
Alibaba), we can assign **Stern** to this shift.
   
2. **12pm-6pm Shift:**
   - For this shift, we can assign the other available guard with a knife qualification, **Alibaba**.
   
3. **6pm-9pm Shift:**
   - Either Stern or Alibaba can cover this shift. Assuming they alternate, assigning **Stern** to ensure each 
guard has an equal number of shifts, or adjust based on their availability or other factors.

### Summary Schedule:

- **6am-12pm:** Stern (qualified with knife)
- **12pm-6pm:** Alibaba (qualified with knife)
- **6pm-9pm:** Stern (qualified with knife) 

This allocation ensures that each shift for Post 1, which requires a knife, is covered by a suitably qualified 
guard.